In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import itertools as it
import multiprocessing
import numpy as np
from multiprocessing import Pool
from joblib import Parallel, delayed
from scipy.stats import mstats as s
import swifter

In [2]:
g=pd.read_csv("GENIE3.all_ranks.ranked_regulator_target_interactions.tsv.gz",delimiter="\t")
g.columns=[x.replace(".","_") for x in g.columns]
g.head()

,regulatoryGene,targetGene,weight,regulator_rank,target_rank,mutual_rank
0,Pp3c17_930.mRNA,Pp3s340_10.mRNA,0.067781,1,1,1.0
1,Pp3c1_27430.pre_miRNA,Pp3c15_12100.mRNA,0.067249,1,1,1.0
2,Pp3c17_17550.mRNA,Pp3c23_130.mRNA,0.066469,1,1,1.0
3,Pp3c17_18920.pre_miRNA,Pp3s32_420.mRNA,0.066327,1,1,1.0
4,Pp3c15_549.mRNA,Pp3c4_24040.mRNA,0.064005,1,1,1.0


In [3]:
len(g)

61897747

In [4]:
v=pd.read_csv("all_genes.vst.tsv.gz",delimiter="\t",index_col=0)
v.columns=[x.replace(".","_") for x in v.columns]
v.head()

,SRR787291,SRR787292,SRR787293,SRR787294,SRR787295,SRR790623,SRR790624,SRR790625,SRR790254,SRR790259,...,65-D14-3,66-LO14,67-LO14-2,68-LO14-3,69-LG14,70-LG14-2,71-LG14-3,72-1-14,73-1-14-2,74-1-14-3
Pp3c1_100.mRNA,9.862156,9.351662,10.623834,8.916050,9.964518,8.777317,8.434408,8.490788,8.997075,8.532412,...,10.256458,8.946873,10.769640,10.293939,9.185842,11.330099,10.749067,8.716056,10.795856,9.818899
Pp3c1_1000.mRNA,7.381567,7.559691,8.580751,7.305779,9.089045,10.578629,10.377033,10.091441,9.826457,9.419218,...,8.284943,8.206360,8.019149,8.240060,8.342942,8.548026,8.676593,7.724645,7.632214,7.874430
Pp3c1_10000.mRNA,10.427010,9.913541,9.623924,9.697720,9.450088,10.555744,9.812155,10.504621,9.839770,9.944695,...,9.518657,9.931283,9.188891,9.936725,9.590335,9.326010,10.018545,10.072072,9.137332,9.567608
Pp3c1_10010.mRNA,7.404392,7.265683,7.924842,7.061676,8.785212,10.227576,10.298046,10.163573,10.220884,9.600121,...,8.642796,8.356125,8.505761,8.541699,8.472045,8.560633,8.811026,8.035877,8.379157,8.341642
Pp3c1_10020.seryl_tRNA,4.192570,4.192570,4.192570,4.192570,4.192570,4.192570,4.192570,4.192570,4.192570,4.192570,...,4.192570,4.192570,4.192570,4.192570,4.192570,4.192570,4.192570,4.192570,4.192570,4.192570


In [ ]:
our = [col for col in v.columns if 'SRR' not in col]
our[0:10]

['1-W3',
 '2-W3-2',
 '3-W3-3',
 '4-D3',
 '5-D3-2',
 '6-D3-3',
 '7-LO3',
 '8-LO3-2',
 '9-LO3-3',
 '10-LG3']

In [ ]:
V=v.loc[:,our]
V.head()

,1-W3,2-W3-2,3-W3-3,4-D3,5-D3-2,6-D3-3,7-LO3,8-LO3-2,9-LO3-3,10-LG3,...,65-D14-3,66-LO14,67-LO14-2,68-LO14-3,69-LG14,70-LG14-2,71-LG14-3,72-1-14,73-1-14-2,74-1-14-3
Pp3c1_100.mRNA,7.793855,8.752263,7.742325,7.810348,8.186421,7.501875,7.797784,8.366783,7.710165,8.308130,...,10.256458,8.946873,10.769640,10.293939,9.185842,11.330099,10.749067,8.716056,10.795856,9.818899
Pp3c1_1000.mRNA,9.925153,9.560368,10.310360,10.247521,9.709732,10.294032,10.063675,9.661584,10.007724,10.099481,...,8.284943,8.206360,8.019149,8.240060,8.342942,8.548026,8.676593,7.724645,7.632214,7.874430
Pp3c1_10000.mRNA,10.415738,10.236311,10.123213,10.187037,9.739825,9.917680,10.276247,10.164440,9.959427,10.491551,...,9.518657,9.931283,9.188891,9.936725,9.590335,9.326010,10.018545,10.072072,9.137332,9.567608
Pp3c1_10010.mRNA,9.955417,9.563961,10.347522,10.056648,9.546546,10.112513,9.908969,9.440217,9.926649,9.854212,...,8.642796,8.356125,8.505761,8.541699,8.472045,8.560633,8.811026,8.035877,8.379157,8.341642
Pp3c1_10020.seryl_tRNA,4.192570,4.192570,4.192570,4.192570,4.192570,4.192570,4.192570,4.192570,4.192570,4.192570,...,4.192570,4.192570,4.192570,4.192570,4.192570,4.192570,4.192570,4.192570,4.192570,4.192570


In [ ]:
def get_correlation (row):
    x=v.loc[row["regulatoryGene"]]
    y=v.loc[row["targetGene"]]
    cor=x.corr(y)
    x=V.loc[row["regulatoryGene"]]
    y=V.loc[row["targetGene"]]
    cor2=x.corr(y)
    return(pd.Series({ "full_cor": cor, "full_reg": "+" if cor >0 else "-", "our_cor": cor2, "our_reg": "+" if cor2 >0 else "-"}))

In [ ]:
g=g.merge(g.swifter.set_npartitions(npartitions=40).set_dask_threshold(dask_threshold=3).progress_bar(False).apply(get_correlation,axis=1),left_index=True, right_index=True)

/home/pgsb/daniel.lang/anaconda3/envs/genie3/lib/python3.6/site-packages/pandas/core/ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


In [9]:
g.head()

,regulatoryGene,targetGene,weight,regulator_rank,target_rank,mutual_rank,full_cor,full_reg,our_cor,our_reg
0,Pp3c17_930.mRNA,Pp3s340_10.mRNA,0.067781,1,1,1.0,0.130485,+,1.874270e-15,+
1,Pp3c1_27430.pre_miRNA,Pp3c15_12100.mRNA,0.067249,1,1,1.0,0.558926,+,6.662628e-01,+
2,Pp3c17_17550.mRNA,Pp3c23_130.mRNA,0.066469,1,1,1.0,0.846470,+,8.916522e-01,+
3,Pp3c17_18920.pre_miRNA,Pp3s32_420.mRNA,0.066327,1,1,1.0,1.000000,+,1.000000e+00,+
4,Pp3c15_549.mRNA,Pp3c4_24040.mRNA,0.064005,1,1,1.0,1.000000,+,1.000000e+00,+


In [10]:
g.to_csv("GENIE3.all_ranks.ranked_regulator_target_interactions.correlations_and_directionality.tsv.gz",sep="\t",index=None)

In [11]:
g.head()
print(g.groupby(["full_reg","our_reg"]).targetGene.count())
print(g.groupby(["full_reg","our_reg"]).full_cor.median())
print(g.groupby(["full_reg","our_reg"]).our_cor.median())

full_reg  our_reg
+         +          19376443
          -          12365192
-         +          13250343
          -          16905769
Name: targetGene, dtype: int64
full_reg  our_reg
+         +          0.139783
          -          0.097219
-         +         -0.069188
          -         -0.104989
Name: full_cor, dtype: float64
full_reg  our_reg
+         +          0.174921
          -         -0.104828
-         +          0.080410
          -         -0.128320
Name: our_cor, dtype: float64
